# Explore the different corpora

In [1]:
import sys
sys.path.append("../python/")

In [2]:
import pentoref.IO as IO

In [11]:
# Get the dataframes (can also use a database method in instead)
dfwords, dfutts, dfrefs, dfscenes, dfactions = IO.convert_subcorpus_raw_data_to_dataframes("../../TAKE_PENTOREF")

r7
/home/dsg-labuser/git/pentoref/TAKE_PENTOREF/derived_data/multimodal_data/scene_information/r7
r6
/home/dsg-labuser/git/pentoref/TAKE_PENTOREF/derived_data/multimodal_data/scene_information/r6
r4
/home/dsg-labuser/git/pentoref/TAKE_PENTOREF/derived_data/multimodal_data/scene_information/r4
r5
/home/dsg-labuser/git/pentoref/TAKE_PENTOREF/derived_data/multimodal_data/scene_information/r5
r2
/home/dsg-labuser/git/pentoref/TAKE_PENTOREF/derived_data/multimodal_data/scene_information/r2
r3
/home/dsg-labuser/git/pentoref/TAKE_PENTOREF/derived_data/multimodal_data/scene_information/r3
r1
/home/dsg-labuser/git/pentoref/TAKE_PENTOREF/derived_data/multimodal_data/scene_information/r1
1137 text files read. 1146 XML files parsed.
7 TextGrids read.
TAKE
Processing:
r4...
r5...
r6...
r7...
r1...
r2...
r3...
Done.


In [13]:
dfwords

,endtime,gameID,position,refID,starttime,uttID,word
0,320.583357,r4_1,1,1,320.249396,1,okay
1,323.103442,r4_1,1,1,322.691291,2,ähm
2,323.379596,r4_1,2,1,323.103442,2,also
3,323.635022,r4_1,3,1,323.379596,2,ich
4,323.750636,r4_1,4,1,323.635022,2,hab
5,324.272242,r4_1,5,1,323.828608,2,mir
6,325.772230,r4_1,1,1,325.586710,3,ein
7,326.143270,r4_1,2,1,325.772230,3,Kreuz
8,327.156780,r4_1,3,1,326.143270,3,ausgesucht
9,327.624739,r4_1,4,1,327.377097,3,äh
